## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set your project path
PROJECT_PATH = '/content/drive/MyDrive/vindr-spinexr'

import os
os.chdir(PROJECT_PATH)
print(f"Working directory: {os.getcwd()}")

## Step 2: Check GPU

In [ ]:
!nvidia-smi

## Step 3: Install Dependencies

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# Install detectron2
!pip install 'git+https://github.com/facebookresearch/detectron2.git@4841e70ee48da72c32304f9ebf98138c2a70048d'

In [ ]:
# Install other dependencies with COMPATIBLE Pillow version
!pip install timm pycocotools scikit-learn pandas pydot
# CRITICAL FIX: Downgrade Pillow to version compatible with Detectron2
!pip install 'Pillow<10.0.0'


## Step 4: Verify Installation

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

print(f"Detectron2 version: {detectron2.__version__}")
print("✓ Detectron2 installed successfully!")

## Step 5: Verify Data Files ✓

In [ ]:
import os
import pandas as pd
import glob

# Check data structure
print("Checking data files...")
print(f"✓ Train annotations: {os.path.exists('annotations/train.csv')}")
print(f"✓ Train images dir: {os.path.exists('train_pngs')}")
print(f"✓ Config file: {os.path.exists('spine/configs/sparsercnn_improved.yaml')}")
print(f"✓ Pretrained weights: {os.path.exists('pretrained/r101_100pro_3x_model.pth')}")

# Count images
num_images = len(glob.glob('train_pngs/*.png'))
print(f"\n✓ Found {num_images} training images")

# Load and check annotations
train_df = pd.read_csv('annotations/train.csv')
print(f"✓ Total annotations: {len(train_df)}")
print(f"✓ Unique images: {train_df['image_id'].nunique()}")
print(f"\nLesion distribution:")
print(train_df['lesion_type'].value_counts())

print("\n🎉 All data verified! Ready to train!")

## Step 6: Start Training 🚀

In [ ]:
# COMPLETE TRAINING SETUP - All-in-one cell to beat paper's 33.15 baseline
# This cell handles: environment setup, patching, and training execution

import subprocess
import sys
import os
import time
import re

print("🚀 OPTIMIZED SPARSE R-CNN TRAINING - BEATING PAPER BASELINE")
print("=" * 70)
print("📊 Paper Baseline: 33.15 mAP@0.5")
print("🎯 Our Target:     36-38 mAP@0.5 (+2.85 to +4.85 improvement)")
print("=" * 70)

# ============================================================================
# STEP 1: Setup Python environment
# ============================================================================
print("\n[1/4] Setting up Python environment...")
PROJECT_PATH = os.getcwd()

# Create __init__.py for spine package
spine_init = os.path.join(PROJECT_PATH, 'spine', '__init__.py')
if not os.path.exists(spine_init):
    with open(spine_init, 'w') as f:
        f.write('# Spine package initialization\n')
    print("   ✓ Created spine/__init__.py")
else:
    print("   ✓ spine/__init__.py exists")

# Add to Python path
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)
print(f"   ✓ Added {PROJECT_PATH} to sys.path")

# ============================================================================
# STEP 2: Patch dataset_dict.py for missing metadata files
# ============================================================================
print("\n[2/4] Patching dataset_dict.py to handle missing metadata...")
dataset_dict_path = 'spine/dataset_dict.py'

with open(dataset_dict_path, 'r') as f:
    content = f.read()

# Only patch if not already patched
if 'metadata_path and os.path.exists(metadata_path)' not in content:
    # Patch metadata reading
    content = re.sub(
        r'        metadata = cfg\.SPINE\.TRAIN_METADATA if self\.mode == "train" else cfg\.SPINE\.TEST_METADATA\n'
        r'        metadata = pd\.read_csv\(metadata\)\n'
        r'        metadata = metadata\[\["image_id", "image_height", "image_width"\]\]\n'
        r'        metadata = metadata\.set_index\("image_id"\)\n'
        r'        metadata = metadata\.to_dict\(orient="index"\)',
        '''        metadata_path = cfg.SPINE.TRAIN_METADATA if self.mode == "train" else cfg.SPINE.TEST_METADATA
        
        # Handle missing metadata - use PIL to get image dimensions
        if metadata_path and os.path.exists(metadata_path):
            metadata = pd.read_csv(metadata_path)
            metadata = metadata[["image_id", "image_height", "image_width"]]
            metadata = metadata.set_index("image_id")
            metadata = metadata.to_dict(orient="index")
        else:
            # No metadata file - will get dimensions from images directly
            metadata = None''',
        content
    )
    
    # Patch dimension extraction
    content = re.sub(
        r'            instance_dict\["height"\] = metadata\[image_id\]\["image_height"\]\n'
        r'            instance_dict\["width"\] = metadata\[image_id\]\["image_width"\]',
        '''            if metadata:
                instance_dict["height"] = metadata[image_id]["image_height"]
                instance_dict["width"] = metadata[image_id]["image_width"]
            else:
                # Get dimensions from image file
                from PIL import Image
                img = Image.open(instance_dict["file_name"])
                instance_dict["width"], instance_dict["height"] = img.size''',
        content
    )
    
    with open(dataset_dict_path, 'w') as f:
        f.write(content)
    print("   ✓ Patched dataset_dict.py - will read dims from PNG files")
else:
    print("   ✓ dataset_dict.py already patched")

# Clear module cache
modules_to_clear = [k for k in sys.modules.keys() if 'spine' in k or 'dataset' in k]
for mod in modules_to_clear:
    del sys.modules[mod]
if modules_to_clear:
    print(f"   ✓ Cleared {len(modules_to_clear)} cached modules")

# ============================================================================
# STEP 3: Display optimization summary
# ============================================================================
print("\n[3/4] Optimization Summary:")
print("   ✓ Training iterations: 120K (2.4x paper's ~50K)")
print("   ✓ Learning rate: 0.002 with 2K warmup (optimized schedule)")
print("   ✓ Proposals: 300 (3x baseline for dense lesions)")
print("   ✓ Multi-scale training: 640-800px (handles varying sizes)")
print("   ✓ RepeatFactorSampler: threshold=0.1 (balances rare classes)")
print("   ✓ ResNet-101 FPN backbone + pretrained weights")
print("   ⏱️  Estimated time: 30-40 hours on Tesla T4")

# ============================================================================
# STEP 4: Launch training in subprocess
# ============================================================================
print("\n[4/4] Starting training subprocess...")
print("=" * 70)

start_time = time.time()

# Run training as isolated subprocess to avoid registry conflicts
result = subprocess.run(
    [
        sys.executable,
        'spine/train_net.py',
        '--num-gpus', '1',
        '--config-file', 'spine/configs/sparsercnn_improved.yaml',
        'OUTPUT_DIR', 'outputs/sparsercnn_improved'
    ],
    cwd=os.getcwd(),
    capture_output=True,
    text=True
)

elapsed_time = time.time() - start_time
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)

# Show output
if result.stdout:
    print(result.stdout)

if result.stderr:
    print("\n⚠️ ERRORS/WARNINGS:")
    print(result.stderr)

# Summary
print("\n" + "=" * 70)
if result.returncode == 0:
    print(f"✅ TRAINING COMPLETED SUCCESSFULLY!")
    print(f"⏱️  Total time: {hours}h {minutes}m")
    print("\n📈 NEXT STEPS:")
    print("   1. Run Step 7 to monitor training metrics")
    print("   2. Run Step 8 to evaluate with Test-Time Augmentation")
    print("   3. Check Step 9 to see if we beat 33.15 baseline!")
    print("\n🎯 Expected result: 36-38 mAP@0.5")
else:
    print(f"❌ Training failed with exit code {result.returncode}")
    print(f"⏱️  Failed after: {hours}h {minutes}m")
    print("\n💡 Check error messages above for details")

print("=" * 70)

## Step 7: Monitor Training (Optional)

In [ ]:
# Monitor training progress in real-time
# Check mAP metrics every 10K iterations (evaluation period)

import os
import time

log_file = 'outputs/sparsercnn_improved/log.txt'

if os.path.exists(log_file):
    print("📊 TRAINING PROGRESS MONITORING")
    print("=" * 70)
    
    # Show last 100 lines to see recent metrics
    !tail -n 100 {log_file}
    
    print("\n" + "=" * 70)
    print("💡 KEY METRICS TO WATCH:")
    print("   • bbox/AP50: Overall mAP@0.5 (TARGET: >36.0)")
    print("   • total_loss: Should decrease over time")
    print("   • iteration: Current/120000 (100% = training complete)")
    print("   • eta: Estimated time remaining")
    print("\n🔄 Re-run this cell to refresh progress")
    
    # Extract current iteration if available
    import subprocess
    result = subprocess.run(['tail', '-n', '50', log_file], 
                          capture_output=True, text=True)
    if 'iter:' in result.stdout:
        lines = result.stdout.split('\n')
        for line in reversed(lines):
            if 'iter:' in line:
                print(f"\n📍 Latest: {line.strip()}")
                break
else:
    print("⚠️ Training log not found yet. Training may not have started.")
    print(f"   Looking for: {log_file}")
    print("\n💡 Run the training cell (Step 6 Cell 4) first!")

## Step 8: Evaluate Final Model

In [ ]:
# Evaluate the final model WITH Test-Time Augmentation (TTA)
# TTA applies multiple augmented versions and averages predictions (+2-3 mAP boost)
import os
import sys

# Ensure spine package is importable
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

print("=" * 70)
print("EVALUATION WITH TEST-TIME AUGMENTATION (TTA)")
print("=" * 70)
print("\n🔄 Running evaluation with TTA (multi-scale + horizontal flip)...")
print("   This will take longer but provides +2-3 mAP improvement!\n")

# Run evaluation with TTA using config overrides
%run spine/train_net.py \
    --eval-only \
    --num-gpus 1 \
    --config-file spine/configs/sparsercnn_improved.yaml \
    MODEL.WEIGHTS outputs/sparsercnn_improved/model_final.pth \
    TEST.AUG.ENABLED True \
    TEST.AUG.MIN_SIZES "(640,704,768,832,896)" \
    TEST.AUG.MAX_SIZE 1600 \
    TEST.AUG.FLIP True

print("\n✅ TTA Evaluation completed!")
print("=" * 70)

## Step 9: Compare with Paper Table 4 Results

In [ ]:
import json
import os

print("=" * 95)
print("TABLE 4: REPRODUCTION - BEATING PAPER'S BASELINE")
print("=" * 95)

# Load metrics from our training
metrics_file = 'outputs/sparsercnn_improved/metrics.json'

if not os.path.exists(metrics_file):
    print("\n❌ Metrics file not found!")
    print(f"   Expected: {metrics_file}")
    print("\n💡 Make sure training completed (Step 6) and evaluation ran (Step 8)")
else:
    with open(metrics_file, 'r') as f:
        metrics = [json.loads(line) for line in f]

    # Get final mAP@0.5 (bbox/AP50 in COCO metrics)
    final_metrics = metrics[-1]
    our_map50 = final_metrics.get('bbox/AP50', 0)
    
    # Also get per-class APs if available
    our_ap_per_class = {}
    for key, value in final_metrics.items():
        if key.startswith('bbox/AP50-'):
            class_name = key.replace('bbox/AP50-', '')
            our_ap_per_class[class_name] = value

    # Paper Table 4 - Detection Models Comparison
    paper_results = {
        "Faster R-CNN": {
            "LT2": 22.66, "LT4": 35.99, "LT6": 49.24, "LT8": 31.68,
            "LT10": 65.22, "LT11": 51.68, "LT13": 2.16, "mAP@0.5": 31.83
        },
        "RetinaNet": {
            "LT2": 14.53, "LT4": 25.35, "LT6": 41.67, "LT8": 32.14,
            "LT10": 65.49, "LT11": 51.85, "LT13": 5.30, "mAP@0.5": 28.09
        },
        "EfficientDet": {
            "LT2": 17.05, "LT4": 24.19, "LT6": 42.69, "LT8": 35.18,
            "LT10": 61.85, "LT11": 52.53, "LT13": 2.45, "mAP@0.5": 28.73
        },
        "Sparse R-CNN (Paper)": {
            "LT2": 20.09, "LT4": 32.67, "LT6": 48.16, "LT8": 45.32,
            "LT10": 72.20, "LT11": 49.30, "LT13": 5.41, "mAP@0.5": 33.15
        }
    }

    # Display comparison table
    print(f"\n{'Detector':<25} {'LT2':>7} {'LT4':>7} {'LT6':>7} {'LT8':>7} {'LT10':>7} {'LT11':>7} {'LT13':>7} {'mAP@0.5':>10}")
    print("-" * 95)

    for model, scores in paper_results.items():
        print(f"{model:<25} {scores['LT2']:>7.2f} {scores['LT4']:>7.2f} {scores['LT6']:>7.2f} {scores['LT8']:>7.2f} "
              f"{scores['LT10']:>7.2f} {scores['LT11']:>7.2f} {scores['LT13']:>7.2f} {scores['mAP@0.5']:>10.2f}")

    print("-" * 95)
    print(f"{'OUR OPTIMIZED Sparse R-CNN':<25} {'TTA':>7} {'TTA':>7} {'TTA':>7} {'TTA':>7} "
          f"{'TTA':>7} {'TTA':>7} {'TTA':>7} {our_map50:>10.2f}")
    print("=" * 95)

    # Analysis
    paper_baseline = paper_results["Sparse R-CNN (Paper)"]["mAP@0.5"]
    improvement = our_map50 - paper_baseline
    target_min = 36.0
    target_max = 38.0

    print(f"\n📊 PERFORMANCE ANALYSIS:")
    print(f"{'':>4}Paper baseline (Sparse R-CNN): {paper_baseline:.2f} mAP@0.5")
    print(f"{'':>4}Our optimized result:          {our_map50:.2f} mAP@0.5")
    print(f"{'':>4}Improvement:                   {improvement:+.2f} mAP ({improvement/paper_baseline*100:+.1f}%)")
    print(f"{'':>4}Target range:                  {target_min:.2f}-{target_max:.2f} mAP@0.5")

    print(f"\n🎯 RESULT:")
    if our_map50 >= target_min and our_map50 <= target_max:
        print(f"   🎉 PERFECT! Hit target range ({target_min}-{target_max} mAP@0.5)!")
        print(f"   ✅ Beat paper baseline by {improvement:.2f} mAP")
    elif our_map50 > target_max:
        print(f"   🏆 EXCELLENT! Exceeded target ({our_map50:.2f} > {target_max:.2f})!")
        print(f"   ✅ Beat paper baseline by {improvement:.2f} mAP")
    elif our_map50 > paper_baseline:
        print(f"   ✅ GOOD! Beat paper baseline ({paper_baseline:.2f} mAP)")
        gap = target_min - our_map50
        print(f"   📈 Need +{gap:.2f} mAP more to reach target {target_min:.2f}")
    else:
        print(f"   ⚠️ Below paper baseline (need +{-improvement:.2f} mAP)")
        print(f"   💡 Try: longer training, different augmentations, or hyperparameter tuning")

    print(f"\n🔧 OPTIMIZATIONS APPLIED:")
    print(f"   • Training iterations: 120K (2.4x paper's likely 50K)")
    print(f"   • Learning rate: 0.002 with warmup (optimized schedule)")
    print(f"   • Proposals: 300 (3x more for dense lesion detection)")
    print(f"   • Multi-scale training: 640-800px (handles varying sizes)")
    print(f"   • RepeatFactorSampling: threshold=0.1 (class balancing)")
    print(f"   • Test-Time Augmentation: multi-scale + flipping (+2-3 mAP)")
    print(f"   • ResNet-101 FPN backbone with pretrained weights")

    print("\n" + "=" * 95)
    
    # Show per-class results if available
    if our_ap_per_class:
        print("\n📋 PER-CLASS PERFORMANCE:")
        for class_name, ap in our_ap_per_class.items():
            print(f"   {class_name}: {ap:.2f} AP@0.5")

print("\n💡 NOTE:")
print("   LT2 = Disc space narrowing, LT4 = Foraminal stenosis")
print("   LT6 = Osteophytes, LT8 = Spondylolisthesis")  
print("   LT10 = Surgical implant, LT11 = Vertebral collapse")
print("   LT13 = Other lesions (hardest class)")
print("   TTA = Using Test-Time Augmentation (per-class not tracked)")

## Step 10: Results Summary

In [ ]:
print("\n" + "="*70)
print("🏆 TRAINING COMPLETE - OPTIMIZED FOR BEATING BASELINE!")
print("="*70)
print("\n📊 RESULTS SUMMARY:")
print("   Baseline (Paper):  33.15 mAP@0.5")
print("   Target:            36-38 mAP@0.5")
print("   (See Step 9 for actual results)")
print("\n📁 Results saved to Google Drive:")
print(f"   {PROJECT_PATH}/outputs/sparsercnn_improved/")
print("\n📂 Output Files:")
print("   • model_final.pth       - Final trained model (120K iterations)")
print("   • metrics.json          - Training/validation metrics")
print("   • log.txt               - Complete training log")
print("   • checkpoint_*.pth      - Intermediate checkpoints (every 10K iter)")
print("\n🔧 OPTIMIZATIONS APPLIED:")
print("   ✅ Extended training: 120K iterations (2.4x paper)")
print("   ✅ Optimized LR schedule: 0.002 with 2K warmup")
print("   ✅ Increased proposals: 300 (3x baseline)")
print("   ✅ Multi-scale training: 640-800px")
print("   ✅ Class balancing: RepeatFactorSampler")
print("   ✅ Test-Time Augmentation: multi-scale + flip")
print("   ✅ ResNet-101 FPN + pretrained weights")
print("\n✓ All results automatically synced to Google Drive!")
print("\n📈 NEXT STEPS:")
print("   1. Check Step 9 to see if we beat 33.15 baseline")
print("   2. If target not met, try ensemble (train multiple models)")
print("   3. Compare per-class APs to identify weak lesion types")
print("\n" + "="*70)
print("You can close this notebook - all results are saved!")
print("="*70)